In [ ]:
!pip install gdown
!pip install tqdm
!pip install ultralytics


import gdown          
import os
import shutil
import zipfile
from google.colab import files 
import random
import itertools
import math
import xgboost as xgb

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from collections import deque

import torch
import torch.nn as nn
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, WeightedRandomSampler, DataLoader, SubsetRandomSampler
from torchvision import transforms
import torchvision.transforms as T
import torch.optim as optim
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

import cv2
from tqdm import tqdm 
from PIL import Image
from IPython.display import display, FileLink, Markdown
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt




url_videos = "https://drive.google.com/drive/folders/1w28fjrEBie5k1MbG14Dw25Zvj_jxJ6cE?usp=sharing"
gdown.download_folder(url_videos, quiet=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


['/content/test/reid_checkpoints/phase1_reid_hybrid_ENDZONE_epoch_20.pth',
 '/content/test/reid_checkpoints/phase1_reid_hybrid_SIDELINE_epoch_20.pth',
 '/content/test/reid_checkpoints/reid_hybrid_ENDZONE_best.pth',
 '/content/test/reid_checkpoints/reid_hybrid_SIDELINE_best.pth',
 '/content/test/best_dualview_96.pth',
 '/content/test/best_weights_players_detection.pt',
 '/content/test/best.pt',
 '/content/test/best2.pt',
 '/content/test/ckpt.t7',
 '/content/test/helmet_dataset.yaml',
 '/content/test/helmets_extracted.zip',
 '/content/test/nfl_r2plus1d_Endzone_BEST_F1.pth',
 '/content/test/nfl_r2plus1d_Endzone_best.pth',
 '/content/test/nfl_r2plus1d_Sideline_BEST_F1.pth',
 '/content/test/nfl_r2plus1d_Sideline_BEST_F1(1).pth',
 '/content/test/nfl_r2plus1d_Sideline_best.pth',
 '/content/test/submission.csv',
 '/content/test/test.zip',
 '/content/test/train_labels.csv',
 '/content/test/xgb_threshold.txt',
 '/content/test/xgboost_impact_model.json']

In [ ]:

original_path = "test"
new_base = "datasets"
new_path = os.path.join(new_base, "Dataset")

os.makedirs(new_base, exist_ok=True)

if not os.path.exists(new_path):
    shutil.move(original_path, new_path)
    print(f" Folder '{original_path}' moved in '{new_path}'")
dataset_path = "datasets/Dataset"

# extract zip files and remove them
def extract_and_remove_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)
    print(f"File zip {zip_path} extracted and removed.")

zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
for zip_file in zip_files:
    zip_path = os.path.join(dataset_path, zip_file)
    extract_and_remove_zip(zip_path, dataset_path)

✅ Cartella 'test' spostata in 'datasets/Dataset'
File zip datasets/Dataset/test.zip estratto e rimosso.
File zip datasets/Dataset/helmets_extracted.zip estratto e rimosso.


# ***Detection YOLO***

In [ ]:
def get_helmet_detections(video_dir, model_path, conf_conf=0.2):


    print(f" Yolo Detection on video in: {video_dir}")

    model = YOLO(model_path)

    all_detections = []

    video_files = [f for f in os.listdir(video_dir) if f.endswith((".mp4", ".avi", ".mov"))]

    for video_name in video_files:
        video_path = os.path.join(video_dir, video_name)
        cap = cv2.VideoCapture(video_path)

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        frame_num = 1

        pbar = tqdm(total=total_frames, desc=f"👀 {video_name}", unit="fr", leave=False)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, verbose=False)[0]

            if results.boxes is not None:
                boxes = results.boxes.xyxy.cpu().numpy()
                confs = results.boxes.conf.cpu().numpy()

                for box, conf in zip(boxes, confs):
                    if conf > conf_conf:
                        x1, y1, x2, y2 = box.astype(int)

                        # Salviamo nella struttura leggera
                        all_detections.append({
                            'video_file': video_name,
                            'frame_num': frame_num,
                            'x1': x1,
                            'y1': y1,
                            'x2': x2,
                            'y2': y2,
                            'conf': float(conf)
                        })

            frame_num += 1
            pbar.update(1)

        cap.release()
        pbar.close()

    df_detections = pd.DataFrame(all_detections)

    if not df_detections.empty:
        df_detections = df_detections.sort_values(by=['video_file', 'frame_num']).reset_index(drop=True)

    print(f"\n Detection complete. Found {len(df_detections)} helmets.")
    return df_detections

VIDEO_DIR = "/content/datasets/Dataset/test"
MODEL_PATH = "/content/datasets/Dataset/best.pt"

df_raw_detections = get_helmet_detections(VIDEO_DIR, MODEL_PATH)

print(df_raw_detections.head())

# ***Sostituzione YOLO con CSV***

In [ ]:
GT = True

if os.path.exists(GT_PATH) and GT:
    # 1. Caricamento del file train_labels.csv
    df_gt = pd.read_csv(GT_PATH)

    # 2. Identificazione dei video da processare
    # Se hai una lista di video specifica (es. quella usata per YOLO), usala qui.
    # Altrimenti, prendiamo i video presenti nella cartella di test.
    test_videos = [f for f in os.listdir(VIDEO_DIR) if f.endswith('.mp4')]

    # 3. Filtraggio GT solo per i video presenti nella cartella di test
    df_raw_detections = df_gt[df_gt['video'].isin(test_videos)].copy()

    if df_raw_detections.empty:
        print("⚠️ ATTENZIONE: Nessuna corrispondenza trovata tra i video in VIDEO_DIR e il file GT!")
    else:
        # 4. Conversione formato: GT (left, top, width, height) -> Pipeline (x1, y1, x2, y2)
        df_raw_detections['x1'] = df_raw_detections['left']
        df_raw_detections['y1'] = df_raw_detections['top']
        df_raw_detections['x2'] = df_raw_detections['left'] + df_raw_detections['width']
        df_raw_detections['y2'] = df_raw_detections['top'] + df_raw_detections['height']

        # 5. Allineamento nomi colonne
        df_raw_detections = df_raw_detections.rename(columns={
            'video': 'video_file',
            'frame': 'frame_num'
        })

        # 6. Pulizia colonne extra per pulizia (manteniamo solo quelle necessarie)
        cols_to_keep = ['video_file', 'frame_num', 'x1', 'y1', 'x2', 'y2']
        # Teniamo anche la label originale (es. H30) per debug avanzato se serve
        if 'label' in df_raw_detections.columns:
            cols_to_keep.append('label')

        df_raw_detections = df_raw_detections[cols_to_keep].reset_index(drop=True)

        print(f"✅ Struttura 'df_raw_detections' aggiornata con {len(df_raw_detections)} box GT.")
        print(f"   Video target rilevati: {df_raw_detections['video_file'].nunique()}")
else:
    if os.path.exists(GT_PATH):
        print(f"❌ ERRORE: File GT non trovato in {GT_PATH}")
    else:
        print("Usiamo le detection di YOLO")

❌ ERRORE: File GT non trovato in /content/datasets/Dataset/train_labels.csv


# ***EXTRACTING CROPS***

In [ ]:
VIDEO_DIR = "/content/datasets/Dataset/test"
OUT_DIR_REID = "/content/datasets/Dataset/temp_crops_reid"       # 64x64
OUT_DIR_CONTEXT = "/content/datasets/Dataset/temp_crops_context" # 128x128 (3x context)

os.makedirs(OUT_DIR_REID, exist_ok=True)
os.makedirs(OUT_DIR_CONTEXT, exist_ok=True)


SIZE_REID = 64
SIZE_CONTEXT = 128
CONTEXT_FACTOR = 3

def extract_crops_from_detections(df_detections, video_dir):
    print(f"Extraction crop from {len(df_detections)} detection YOLO...")

    df_detections['crop_path_reid'] = None
    df_detections['crop_path_context'] = None

    unique_videos = df_detections['video_file'].unique()

    for video_name in tqdm(unique_videos, desc="Estrazione Video"):
        video_path = os.path.join(video_dir, video_name)
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"⚠️ Impossibile aprire {video_name}")
            continue

        vid_dets = df_detections[df_detections['video_file'] == video_name].copy().sort_values('frame_num')
        frames_with_dets = vid_dets.groupby('frame_num')

        for frame_idx, group in frames_with_dets:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx - 1)
            ret, frame = cap.read()

            if not ret:
                continue

            h_img, w_img = frame.shape[:2]

            for idx, row in group.iterrows():
                x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])

                x1_c = max(0, x1); y1_c = max(0, y1)
                x2_c = min(w_img, x2); y2_c = min(h_img, y2)

                crop_tight = frame[y1_c:y2_c, x1_c:x2_c]

                if crop_tight.size == 0: continue

                crop_tight_resized = cv2.resize(crop_tight, (SIZE_REID, SIZE_REID))
                filename = f"{video_name.replace('.mp4','')}_{frame_idx}_{idx}.jpg"

                path_reid = os.path.join(OUT_DIR_REID, filename)
                cv2.imwrite(path_reid, crop_tight_resized)

                w_box = x2 - x1
                h_box = y2 - y1
                cx = x1 + w_box / 2
                cy = y1 + h_box / 2

                w_amp = w_box * CONTEXT_FACTOR
                h_amp = h_box * CONTEXT_FACTOR

                x1_amp = int(cx - w_amp / 2)
                y1_amp = int(cy - h_amp / 2)
                x2_amp = int(cx + w_amp / 2)
                y2_amp = int(cy + h_amp / 2)

                x1_amp = max(0, x1_amp); y1_amp = max(0, y1_amp)
                x2_amp = min(w_img, x2_amp); y2_amp = min(h_img, y2_amp)

                crop_context = frame[y1_amp:y2_amp, x1_amp:x2_amp]

                if crop_context.size > 0:
                    # Resize 128x128
                    crop_context_resized = cv2.resize(crop_context, (SIZE_CONTEXT, SIZE_CONTEXT))

                    path_context = os.path.join(OUT_DIR_CONTEXT, filename)
                    cv2.imwrite(path_context, crop_context_resized)

                    df_detections.at[idx, 'crop_path_reid'] = path_reid
                    df_detections.at[idx, 'crop_path_context'] = path_context
                    df_detections.at[idx, 'x1_amp'] = x1_amp
                    df_detections.at[idx, 'y1_amp'] = y1_amp
                    df_detections.at[idx, 'x2_amp'] = x2_amp
                    df_detections.at[idx, 'y2_amp'] = y2_amp

        cap.release()
    return df_detections

if 'df_raw_detections' in locals() and not df_raw_detections.empty:
    df_with_crops = extract_crops_from_detections(df_raw_detections, VIDEO_DIR)


    df_with_crops = df_with_crops.dropna(subset=['crop_path_reid', 'crop_path_context'])

    print("\nData Ready for Tracking:")
    print(df_with_crops[['video_file', 'frame_num', 'crop_path_reid']].head())
else:
    print(" Error")

# ***TRACKING***

In [ ]:

ROOT_DIR = "/content/datasets/Dataset"
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "reid_checkpoints")
VIDEO_DIR = "/content/datasets/Dataset/test"

MAX_COSINE_DISTANCE = 0.70
MAX_IOU_DISTANCE = 0.85
WEIGHT_REID = 0.30

WEIGHT_IOU = 0.70

MAX_AGE = 150
REID_THRESHOLD_REACQ = 0.60

N_INIT = 3
MIN_TRACK_LENGTH = 15

FRAME_W, FRAME_H = 1280, 720
EMBEDDING_DIM = 128
POSITIONAL_FEATURES_DIM = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EXTRACTORS = {}
TRACK_EMBEDDINGS_CACHE = defaultdict(lambda: defaultdict(dict))


class HybridFrameReIDModel(nn.Module):
    def __init__(self, embedding_dim=128, positional_features_dim=4):
        super().__init__()
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(*(list(resnet.children())[:-2]))
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        VISUAL_FEATURE_DIM = 512
        COMBINED_DIM = VISUAL_FEATURE_DIM + positional_features_dim
        self.embedding_layer = nn.Linear(COMBINED_DIM, embedding_dim)

    def forward(self, x_image, x_positional):
        x_image = self.features(x_image)
        x_image = self.avgpool(x_image)
        x_image = torch.flatten(x_image, 1)
        combined_features = torch.cat((x_image, x_positional), dim=1)
        embedding = self.embedding_layer(combined_features)
        return nn.functional.normalize(embedding, p=2, dim=1)

transform = transforms.Compose([
    transforms.ToPILImage(), transforms.Resize((64, 64)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomFeatureExtractor:
    def __init__(self, reid_model, transform, device, frame_w, frame_h):
        self.reid_model = reid_model
        self.transform = transform
        self.device = device
        self.FRAME_W, self.FRAME_H = frame_w, frame_h
        self.reid_model.eval()

    def __call__(self, img_crops, bbox_xywh_abs):
        if len(img_crops) == 0:
            return np.empty((0, self.reid_model.embedding_layer.out_features), dtype=np.float32)
        image_tensors = [self.transform(cv2.cvtColor(c, cv2.COLOR_BGR2RGB)) if c is not None and c.size > 0
                         else torch.zeros(3, 64, 64) for c in img_crops]
        image_batch = torch.stack(image_tensors).to(self.device)
        pos_batch = torch.tensor(bbox_xywh_abs / [self.FRAME_W, self.FRAME_H, self.FRAME_W, self.FRAME_H],
                                 dtype=torch.float32).to(self.device)
        with torch.no_grad():
            return self.reid_model(image_batch, pos_batch).cpu().numpy()

def get_feature_extractor(view_type):
    global EXTRACTORS
    if view_type in EXTRACTORS: return EXTRACTORS[view_type]
    model = HybridFrameReIDModel(EMBEDDING_DIM, POSITIONAL_FEATURES_DIM).to(device)
    ckpt = "reid_hybrid_SIDELINE_best.pth" if 'sideline' in view_type.lower() else "reid_hybrid_ENDZONE_best.pth"
    path = os.path.join(CHECKPOINT_DIR, ckpt)
    if os.path.exists(path):
        sd = torch.load(path, map_location=device)
        model.load_state_dict(sd['model_state_dict'] if 'model_state_dict' in sd else sd, strict=False)
        print(f"ℹ️ Estrattore {view_type} caricato.")
    EXTRACTORS[view_type] = CustomFeatureExtractor(model, transform, device, FRAME_W, FRAME_H)
    return EXTRACTORS[view_type]

class SimpleTrack:
    _count = 0
    def __init__(self, embedding, bbox_abs, frame_num):
        SimpleTrack._count += 1
        self.track_id = SimpleTrack._count
        self.last_bbox_abs = bbox_abs.copy()
        self.embeddings = deque([embedding], maxlen=30)
        self.hits, self.time_since_update = 1, 0
        self.deleted = False

    def update(self, embedding, bbox_abs):
        self.embeddings.append(embedding)
        self.last_bbox_abs, self.hits, self.time_since_update = bbox_abs.copy(), self.hits + 1, 0

    def mark_missed(self):
        self.time_since_update += 1
        if self.time_since_update > MAX_AGE: self.deleted = True

    def is_deleted(self): return self.deleted

    @property
    def predicted_bbox_tlbr(self):
        xc, yc, w, h = self.last_bbox_abs
        return np.array([xc - w/2, yc - h/2, xc + w/2, yc + h/2])

    @property
    def mean_embedding(self):
        m = np.mean(self.embeddings, axis=0)
        return (m / (np.linalg.norm(m) + 1e-6)).reshape(1, -1)


def bbox_iou_simple(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2]-boxA[0]) * (boxA[3]-boxA[1])
    areaB = (boxB[2]-boxB[0]) * (boxB[3]-boxB[1])
    union = float(areaA + areaB - inter)
    return inter / union if union > 0 else 0.0

def associate_detections_to_tracks(all_tracks, new_detections_data, extractor):
    active_tracks = [t for t in all_tracks if not t.is_deleted() and t.time_since_update < 5]
    lost_tracks = [t for t in all_tracks if not t.is_deleted() and t.time_since_update >= 5]

    if not all_tracks or not new_detections_data:
        return [], [t for t in all_tracks if not t.is_deleted()], list(range(len(new_detections_data)))

    det_crops = [d[1] for d in new_detections_data]
    det_boxes_abs = np.vstack([d[2] for d in new_detections_data])
    det_embs = extractor(det_crops, det_boxes_abs)

    matches = []
    unmatched_dets = list(range(len(new_detections_data)))

    if active_tracks:
        track_boxes = np.array([t.predicted_bbox_tlbr for t in active_tracks])
        track_embs = np.vstack([t.mean_embedding for t in active_tracks])

        sim_matrix = cosine_similarity(det_embs, track_embs)
        reid_cost = 1.0 - sim_matrix

        det_boxes_tlbr = np.array([[d[0][0], d[0][1], d[0][0]+d[0][2], d[0][1]+d[0][3]] for d in new_detections_data])
        iou_cost = np.array([[1.0 - bbox_iou_simple(d, t) for t in track_boxes] for d in det_boxes_tlbr])

        cost_matrix = (WEIGHT_REID * reid_cost) + (WEIGHT_IOU * iou_cost)
        cost_matrix[iou_cost > MAX_IOU_DISTANCE] = 1.0

        temp_dets = unmatched_dets.copy()
        while temp_dets and active_tracks:
            sub = cost_matrix[np.ix_(temp_dets, range(len(active_tracks)))]
            i, j = np.unravel_index(sub.argmin(), sub.shape)
            if sub[i, j] > MAX_COSINE_DISTANCE: break

            det_idx = temp_dets.pop(i)
            matches.append((det_idx, active_tracks[j]))
            unmatched_dets.remove(det_idx)

    if lost_tracks and unmatched_dets:
        lost_embs = np.vstack([t.mean_embedding for t in lost_tracks])
        remaining_det_embs = det_embs[unmatched_dets]

        reacq_sim = cosine_similarity(remaining_det_embs, lost_embs)

        for i, det_idx in enumerate(unmatched_dets.copy()):
            best_trk_idx = reacq_sim[i].argmax()
            if reacq_sim[i][best_trk_idx] > (1.0 - REID_THRESHOLD_REACQ):
                matches.append((det_idx, lost_tracks[best_trk_idx]))
                unmatched_dets.remove(det_idx)

    return matches, [t for t in all_tracks if not t.is_deleted()], unmatched_dets


def run_tracking_with_logs(video_name, df_video_crops):
    view_type = 'SIDELINE' if 'Sideline' in video_name else 'ENDZONE'
    extractor = get_feature_extractor(view_type)
    video_key = video_name.replace('.mp4', '')
    all_tracks, tracking_records = [], []
    frames_group = df_video_crops.sort_values('frame_num').groupby('frame_num')

    for frame_idx, frame_data in frames_group:
        new_dets = []
        for idx, row in frame_data.iterrows():
            x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])
            crop = cv2.imread(row['crop_path_reid'])
            if crop is not None:
                w, h = x2 - x1, y2 - y1
                new_dets.append((np.array([x1, y1, w, h]), crop, np.array([x1+w/2, y1+h/2, w, h]), idx))

        matches, current_active_tracks, unmatched = associate_detections_to_tracks(all_tracks, new_dets, extractor)

        matched_set = set()
        for d_idx, track in matches:
            bbox, crop, bbox_abs, orig_idx = new_dets[d_idx]
            emb = extractor([crop], np.array([bbox_abs]))[0]
            track.update(emb, bbox_abs)
            matched_set.add(track)
            tracking_records.append({
                'video_id': video_key, 'frame_num': frame_idx, 'track_id': track.track_id,
                'x_min': int(bbox[0]), 'y_min': int(bbox[1]), 'width': int(bbox[2]), 'height': int(bbox[3]),
                'crop_path_context': df_video_crops.loc[orig_idx, 'crop_path_context']
            })

        for d_idx in unmatched:
            bbox, crop, bbox_abs, orig_idx = new_dets[d_idx]
            emb = extractor([crop], np.array([bbox_abs]))[0]
            nt = SimpleTrack(emb, bbox_abs, frame_idx)
            all_tracks.append(nt)
            tracking_records.append({
                'video_id': video_key, 'frame_num': frame_idx, 'track_id': nt.track_id,
                'x_min': int(bbox[0]), 'y_min': int(bbox[1]), 'width': int(bbox[2]), 'height': int(bbox[3]),
                'crop_path_context': df_video_crops.loc[orig_idx, 'crop_path_context']
            })

        for t in all_tracks:
            if t not in matched_set: t.mark_missed()

    return pd.DataFrame(tracking_records)

if 'df_with_crops' in locals() and not df_with_crops.empty:
    all_tracking_results = []
    video_list = df_with_crops['video_file'].unique()

    for i, video_name in enumerate(video_list):
        print(f"\nProcessing {i+1}/{len(video_list)}: {video_name}")
        df_video_subset = df_with_crops[df_with_crops['video_file'] == video_name].copy()
        df_track_raw = run_tracking_with_logs(video_name, df_video_subset)

        if not df_track_raw.empty:
            track_lengths = df_track_raw.groupby('track_id')['frame_num'].nunique()
            df_track_raw = df_track_raw[df_track_raw['track_id'].isin(track_lengths[track_lengths >= MIN_TRACK_LENGTH].index)].copy()
            final_count = df_track_raw['track_id'].nunique()
            print(f" Tracking complete. Final Tracks: {final_count}")
            all_tracking_results.append(df_track_raw)

    if all_tracking_results:
        final_tracks_df = pd.concat(all_tracking_results, ignore_index=True)
        final_tracks_df.to_csv("final_test_tracks_tracking_log.csv", index=False)
        files.download("final_test_tracks_tracking_log.csv")
        final_consolidated_df = final_tracks_df

# ***GENERATE SEQUENCES***

In [ ]:
GT_PATH = "/content/datasets/Dataset/train_labels.csv"    
IOU_THRESHOLD = 0.35                     
FRAME_TOLERANCE = 4                      # +/- 4 frame

def bbox_iou(box1, box2):
    x1 = max(box1['left'], box2['left'])
    y1 = max(box1['top'], box2['top'])
    x2 = min(box1['left'] + box1['width'], box2['left'] + box2['width'])
    y2 = min(box1['top'] + box1['height'], box2['top'] + box2['height'])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    area1 = box1['width'] * box1['height']
    area2 = box2['width'] * box2['height']

    union = area1 + area2 - intersection

    if union <= 0: return 0.0
    return intersection / union

def evaluate_performance(pred_csv, gt_csv):
    preds = pd.read_csv(pred_csv)
    gt = pd.read_csv(gt_csv)
    test_videos = preds['video'].unique()
    gt = gt[gt['video'].isin(test_videos)].copy()

    if 'confidence' in gt.columns:
        gt = gt[(gt['impact'] == 1) & (gt['confidence'] > 1) & (gt['visibility'] > 0)]
    else:
        gt = gt[gt['impact'] == 1]

    gt['matched'] = False
    preds['is_tp'] = False

    video_results = []

    for video in test_videos:
        v_preds = preds[preds['video'] == video].copy()
        v_gt = gt[gt['video'] == video].copy()

        v_tp = 0
        v_fp = 0

        if 'score' in v_preds.columns:
            v_preds = v_preds.sort_values('score', ascending=False)

        for idx, pred_row in v_preds.iterrows():
            candidates = v_gt[
                (v_gt['frame'] >= pred_row['frame'] - FRAME_TOLERANCE) &
                (v_gt['frame'] <= pred_row['frame'] + FRAME_TOLERANCE) &
                (v_gt['matched'] == False)
            ]

            best_iou = 0
            best_match_idx = -1
            pred_box = {'left': pred_row['left'], 'top': pred_row['top'],
                        'width': pred_row['width'], 'height': pred_row['height']}

            for gt_idx, gt_row in candidates.iterrows():
                gt_box = {'left': gt_row['left'], 'top': gt_row['top'],
                          'width': gt_row['width'], 'height': gt_row['height']}

                iou = bbox_iou(pred_box, gt_box) 
                if iou > best_iou:
                    best_iou = iou
                    best_match_idx = gt_idx

            if best_iou >= IOU_THRESHOLD:
                v_tp += 1
                v_gt.at[best_match_idx, 'matched'] = True
                gt.at[best_match_idx, 'matched'] = True
            else:
                v_fp += 1

        v_fn = len(v_gt[v_gt['matched'] == False])

        v_precision = v_tp / (v_tp + v_fp) if (v_tp + v_fp) > 0 else 0.0
        v_recall = v_tp / (v_tp + v_fn) if (v_tp + v_fn) > 0 else 0.0
        v_f1 = 2 * (v_precision * v_recall) / (v_precision + v_recall) if (v_precision + v_recall) > 0 else 0.0

        video_results.append({
            'Video': video,
            'GT_Count': len(v_gt),
            'Pred_Count': len(v_preds),
            'TP': v_tp, 'FP': v_fp, 'FN': v_fn,
            'Precision': round(v_precision, 4),
            'Recall': round(v_recall, 4),
            'F1': round(v_f1, 4)
        })

    results_df = pd.DataFrame(video_results)

    total_tp = results_df['TP'].sum()
    total_fp = results_df['FP'].sum()
    total_fn = results_df['FN'].sum()

    global_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    global_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    global_f1 = 2 * (global_precision * global_recall) / (global_precision + global_recall) if (global_precision + global_recall) > 0 else 0.0

    print("\n" + "="*85)
    print(f"{'VIDEO RESULTS':^85}")
    print("="*85)
    print(results_df.to_string(index=False))
    print("-" * 85)
    print(f"{'OVERALL':<30} TP: {total_tp:<5} FP: {total_fp:<5} FN: {total_fn:<5}")
    print(f"{'':<30} Precision: {global_precision:.4f} Recall: {global_recall:.4f} F1: {global_f1:.4f}")
    print("="*85)

    return results_df

def extract_inference_sequences(df_tracks):
    print("\n Start extraction for Testing...")

    print(f"   Column in DataFrame: {list(df_tracks.columns)}")

    if 'x_min' in df_tracks.columns and 'left' not in df_tracks.columns:
        df_tracks['left'] = df_tracks['x_min']
    if 'y_min' in df_tracks.columns and 'top' not in df_tracks.columns:
        df_tracks['top'] = df_tracks['y_min']
    if 'width' not in df_tracks.columns:
        if 'x_max' in df_tracks.columns:
            df_tracks['width'] = df_tracks['x_max'] - df_tracks['left']
        else:
            raise KeyError("can't compute width.")

    if 'height' not in df_tracks.columns:
        if 'y_max' in df_tracks.columns:
            df_tracks['height'] = df_tracks['y_max'] - df_tracks['top']
        else:
            raise KeyError("can't compute height")

    SAMPLE_LENGTH = 16
    FRAME_STRIDE = 4
    MAX_SPAN_ALLOWED = 25

    inference_samples = []

    df_tracks = df_tracks.sort_values(by=['video_id', 'track_id', 'frame_num'])

    grouped = df_tracks.groupby(['video_id', 'track_id'])

    for (video_id, track_id), group in tqdm(grouped, desc="Sequencing Tracks"):

        group = group.reset_index(drop=True)
        frames = group['frame_num'].values

        if len(frames) < SAMPLE_LENGTH:
            continue

        view_type = 'Endzone' if 'Endzone' in video_id else 'Sideline'

        for i in range(0, len(frames) - SAMPLE_LENGTH + 1, FRAME_STRIDE):

            sub_seq_df = group.iloc[i : i + SAMPLE_LENGTH]

            start_frame = sub_seq_df['frame_num'].iloc[0]
            end_frame = sub_seq_df['frame_num'].iloc[-1]
            real_span = end_frame - start_frame

            if real_span > MAX_SPAN_ALLOWED:
                continue

            seq_meta_data = []

            for _, row in sub_seq_df.iterrows():
                full_path = row['crop_path_context']
                if pd.isna(full_path):
                    print(f"⚠️ Warning: missing path for track {track_id} frame {row['frame_num']}")
                    continue

                seq_meta_data.append({
                    'file_name': os.path.basename(str(full_path)),
                    'full_path': str(full_path),
                    'frame': row['frame_num'],
                    'left': row['left'],
                    'top': row['top'],
                    'width': row['width'],
                    'height': row['height']
                })

            if len(seq_meta_data) == SAMPLE_LENGTH:
                unique_key = f"{video_id}_{track_id}_{start_frame}"

                inference_samples.append({
                    'key': unique_key,
                    'video_id': video_id,
                    'track_id': track_id,
                    'view': view_type,
                    'data': seq_meta_data,
                    'start_frame': start_frame
                })

    print(f"   Total Sequences: {len(inference_samples)}")

    return inference_samples

if 'final_consolidated_df' in locals() and not final_consolidated_df.empty:
    test_sequences = extract_inference_sequences(final_consolidated_df)

    test_samples_endzone = [s for s in test_sequences if s['view'] == 'Endzone']
    test_samples_sideline = [s for s in test_sequences if s['view'] == 'Sideline']

    print(f"   - Endzone Samples: {len(test_samples_endzone)}")
    print(f"   - Sideline Samples: {len(test_samples_sideline)}")
else:
    print(" Error.")

In [ ]:
DF_TO_CHECK = final_consolidated_df 
VIDEO_DIR = "/content/datasets/Dataset/test"

def diagnose_helmet_density_and_duplicates(df):

    if df.empty:
        print("❌ CRITICO: DataFrame vuoto.")
        return

    # 1. Standardizzazione Nomi (lavoriamo su una copia per sicurezza)
    df_check = df.copy()

    # Rinomina frame
    if 'frame_num' in df_check.columns:
        df_check = df_check.rename(columns={'frame_num': 'frame'})

    # Rinomina coordinate in formato standard per il check
    rename_map = {'x_min': 'left', 'y_min': 'top', 'x_max': 'right', 'y_max': 'bottom'}
    df_check = df_check.rename(columns={k:v for k,v in rename_map.items() if k in df_check.columns})

    # Determiniamo quali colonne usare per definire la "posizione"
    # Di solito bastano left, top, width, height (o right, bottom)
    coord_cols = [c for c in ['left', 'top', 'width', 'height', 'right', 'bottom'] if c in df_check.columns]

    if not coord_cols:
        print("❌ ERRORE: Non trovo colonne di coordinate (left, top, x_min, etc).")
        print(f"   Colonne disponibili: {df_check.columns.tolist()}")
        return

    print(f"ℹ️ Usando colonne coordinate per il check: {coord_cols}")

    # 2. Loop per Video
    unique_videos = df_check['video_id'].unique()

    for video_id in unique_videos:
        print(f"\n🎥 VIDEO: {video_id}")

        # Filtriamo i dati
        vid_df = df_check[df_check['video_id'] == video_id]

        # --- A. CHECK DENSITÀ (Conteggio Caschi) ---
        frame_counts = vid_df.groupby('frame').size()

        avg_helmets = frame_counts.mean()
        max_helmets = frame_counts.max()

        # --- B. CHECK DUPLICATI POSIZIONALI (Il test richiesto) ---
        # Cerchiamo righe che hanno STESSO frame e STESSE coordinate
        # keep=False marca TUTTI i duplicati (es. se ho 3 righe uguali, sono 3 duplicati)
        subset_cols = ['frame'] + coord_cols
        duplicates = vid_df.duplicated(subset=subset_cols, keep=False)
        num_dupes = duplicates.sum()

        # Analisi approfondita frame per frame
        frames_with_stacking = []
        for frame, group in vid_df.groupby('frame'):
            # Quanti box totali in questo frame?
            total_boxes = len(group)
            # Quante posizioni UNICHE in questo frame?
            unique_positions = group.drop_duplicates(subset=coord_cols).shape[0]

            # Se Box Totali > 1 ma Posizioni Uniche == 1, abbiamo lo STACKING TOTALE
            if total_boxes > 1 and unique_positions == 1:
                frames_with_stacking.append(frame)
            # Se ci sono meno posizioni che box, c'è sovrapposizione parziale
            elif unique_positions < total_boxes:
                pass

        # --- REPORTING ---

        # Info Video Reale
        video_path = os.path.join(VIDEO_DIR, video_id + ".mp4")
        cap = cv2.VideoCapture(video_path)
        total_frames_vid = "???"
        if cap.isOpened():
            total_frames_vid = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()

        print(f"   📊 Densità:")
        print(f"      Media Box/Frame:      {avg_helmets:.2f}")
        print(f"      Frame Totali Video:   {total_frames_vid}")
        print(f"      Frame con Dati:       {len(frame_counts)}")

        print(f"   🔍 Analisi Posizioni:")
        if num_dupes == 0:
            print("      ✅ NESSUN DUPLICATO: Tutti i box hanno coordinate distinte.")
        else:
            print(f"      ⚠️ TROVATI {num_dupes} BOX CON POSIZIONI DUPLICATE!")
            print(f"         (Significa che più ID occupano esattamente lo stesso pixel)")

        if len(frames_with_stacking) > 0:
            print(f"      🚨 BUG STACKING RILEVATO in {len(frames_with_stacking)} frame!")
            print(f"         Esempio Frame {frames_with_stacking[0]}: Tutti i box sono sovrapposti.")
            print("         -> I dati di tracking sono corrotti (coordinate copiate).")
        else:
            print("      ✅ Nessun frame con 'stacking totale' (tutti su uno).")

        # Verdetto Finale
        if len(frames_with_stacking) > 0:
            print("      ❌ STATO: DATI CORROTTI (STACKING)")
        elif avg_helmets < 5:
            print("      ⚠️ STATO: DENSITÀ BASSA")
        else:
            print("      ✅ STATO: DATI OK")

# Esegui
if 'final_consolidated_df' in locals():
    diagnose_helmet_density_and_duplicates(final_consolidated_df)
else:
    print("❌ Esegui prima il tracking.")

🕵️‍♂️ DIAGNOSTICA COMPLETA: DENSITÀ E DUPLICATI POSIZIONALI
ℹ️ Usando colonne coordinate per il check: ['left', 'top', 'width', 'height']

🎥 VIDEO: 57906_000718_Endzone
   📊 Densità:
      Media Box/Frame:      20.21
      Frame Totali Video:   434
      Frame con Dati:       434
   🔍 Analisi Posizioni:
      ✅ NESSUN DUPLICATO: Tutti i box hanno coordinate distinte.
      ✅ Nessun frame con 'stacking totale' (tutti su uno).
      ✅ STATO: DATI OK

🎥 VIDEO: 57906_000718_Sideline
   📊 Densità:
      Media Box/Frame:      18.82
      Frame Totali Video:   440
      Frame con Dati:       440
   🔍 Analisi Posizioni:
      ✅ NESSUN DUPLICATO: Tutti i box hanno coordinate distinte.
      ✅ Nessun frame con 'stacking totale' (tutti su uno).
      ✅ STATO: DATI OK

🎥 VIDEO: 57995_000109_Endzone
   📊 Densità:
      Media Box/Frame:      19.07
      Frame Totali Video:   529
      Frame con Dati:       529
   🔍 Analisi Posizioni:
      ✅ NESSUN DUPLICATO: Tutti i box hanno coordinate distinte.
 

# ***DATASET AND MODEL DEFINITION***

In [ ]:
def get_inference_transforms():
    return T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])
    ])

class NFLInferenceDataset(Dataset):
    def __init__(self, samples, root_dir=None, transform=None):
      
        self.samples = samples
        self.root_dir = root_dir
        self.transform = transform

        
        if self.transform is None:
            self.transform = get_inference_transforms()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        meta_data = sample['data'] 

        images = []

        for item in meta_data:
            
            if 'full_path' in item:
                img_path = item['full_path']
            else:
                
                vid_folder = item.get('video_folder', '').replace('.mp4', '')
                fname = item.get('file_name', '')
                if self.root_dir:
                    img_path = os.path.join(self.root_dir, vid_folder, fname)
                else:
                    img_path = fname 

            try:
                
                img = Image.open(img_path).convert('RGB')
                if img.size != (64, 64):
                     img = img.resize((64, 64))

            except Exception:
                
                img = Image.new('RGB', (64, 64), (0, 0, 0))

            img_t = self.transform(img)
            images.append(img_t)

        video_tensor = torch.stack(images).permute(1, 0, 2, 3)

        return video_tensor, idx

In [ ]:
class NFLImpactModel(nn.Module):
    def __init__(self, num_frames=16, hidden_dim=256, dropout_prob=0.5):
        super(NFLImpactModel, self).__init__()

        weights = R2Plus1D_18_Weights.DEFAULT
        self.backbone = r2plus1d_18(weights=weights)

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Linear(in_features, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(hidden_dim, num_frames)
        )

    def forward(self, x):
        features = self.backbone(x)
        out = self.classifier(features)
        return out


# ***INFERENCE STEP***

In [ ]:
FINAL_CONFIG = {
    'Endzone': {
        'thresh': 0.63,       
        'ratio': 0.45,        
        'nms': 6            
    },
    'Sideline': {
        'thresh': 0.66,       
        'ratio': 0.35,        
        'nms': 12            
    }
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GT_PATH = "/content/datasets/Dataset/train_labels.csv"

print(f" Configuration: {FINAL_CONFIG}")

def run_inference_with_consensus(view_name, samples_list, model_path):

    cfg = FINAL_CONFIG[view_name]
    threshold = cfg['thresh']
    consensus_ratio = cfg['ratio']

    print(f"\n Inference {view_name} -> Model: {model_path}")
    print(f"   Parameters -> threshold: {threshold} | Ratio: {consensus_ratio}")

    model = NFLImpactModel(num_frames=16, hidden_dim=256).to(DEVICE)

    try:
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    except RuntimeError as e:
        print(f" ERROR: {e}")
        return []

    model.eval()

    loader = DataLoader(NFLInferenceDataset(samples_list), batch_size=32, shuffle=False)
    detection_map = {}

    with torch.no_grad():
        for video_tensors, indices in tqdm(loader, desc=f"Predicting {view_name}"):
            video_tensors = video_tensors.to(DEVICE)
            probs = torch.sigmoid(model(video_tensors)).cpu().numpy()

            for i, prob_vec in enumerate(probs):
                sample_meta = samples_list[indices[i].item()]
                for off, score in enumerate(prob_vec):
                    f_data = sample_meta['data'][off]
                    key = (sample_meta['video_id'], sample_meta['track_id'], f_data['frame'])

                    if key not in detection_map:
                        detection_map[key] = {'s': [], 'm': f_data}
                    detection_map[key]['s'].append(score)

    refined = []
    for (vid, trk, frm), data in detection_map.items():
        scores = data['s']
        pos_votes = sum(1 for s in scores if s > threshold)
        total_votes = len(scores)

        if total_votes > 0 and (pos_votes / total_votes) >= consensus_ratio:
            parts = vid.split('_')
            refined.append({
                'gameKey': int(parts[0]),
                'playID': int(parts[1]),
                'view': parts[2],
                'video': f"{vid}.mp4",
                'frame': frm,
                'left': int(data['m']['left']),
                'width': int(data['m']['width']),
                'top': int(data['m']['top']),
                'height': int(data['m']['height']),
                'label': f"Player{trk}",
                'score': float(np.mean(scores))
            })

    print(f"   -> Detection: {len(refined)}")
    return refined

def apply_asymmetric_nms(df_preds):
    if df_preds.empty: return df_preds
    print(f" Using NMS on {len(df_preds)} predictions...")
    final_predictions = []

    for view_type in ['Endzone', 'Sideline']:
        view_window = FINAL_CONFIG[view_type]['nms']
        view_df = df_preds[df_preds['view'] == view_type].copy()
        if view_df.empty: continue

        grouped = view_df.groupby(['video', 'label'])
        for (video, label), group in grouped:
            group = group.sort_values('score', ascending=False)
            kept_frames = []
            for _, row in group.iterrows():
                if not any(abs(row['frame'] - kf) < view_window for kf in kept_frames):
                    final_predictions.append(row)
                    kept_frames.append(row['frame'])

    df_final = pd.DataFrame(final_predictions)
    if not df_final.empty: df_final = df_final.sort_values(by=['video', 'frame'])
    print(f" NMS Completed: {len(df_preds)} -> {len(df_final)} impatti.")
    return df_final

def evaluate_performance_robust(pred_csv, gt_csv):

    if isinstance(pred_csv, str): preds = pd.read_csv(pred_csv)
    else: preds = pred_csv.copy()

    gt = pd.read_csv(gt_csv)

    target_videos = [
        "57906_000718_Endzone.mp4", "57906_000718_Sideline.mp4",
        "57995_000109_Endzone.mp4", "57995_000109_Sideline.mp4",
        "58102_002798_Endzone.mp4", "58102_002798_Sideline.mp4"
    ]
    gt = gt[gt['video'].isin(target_videos) & (gt['impact'] == 1) & (gt['visibility'] > 0)].copy()
    gt['matched'] = False

    video_results = []
    FRAME_TOL = 4
    IOU_THRESH = 0.35

    for video in target_videos:
        v_preds = preds[preds['video'] == video].copy() if not preds.empty else pd.DataFrame()
        v_gt = gt[gt['video'] == video].copy()

        v_tp = 0
        v_fp = 0

        if not v_preds.empty:
            v_preds = v_preds.sort_values('score', ascending=False)

            for _, p in v_preds.iterrows():
                candidates = v_gt[
                    (v_gt['frame'].between(p['frame'] - FRAME_TOL, p['frame'] + FRAME_TOL)) &
                    (v_gt['matched'] == False)
                ]

                best_iou = 0
                best_idx = -1

                p_box = [p['left'], p['top'], p['width'], p['height']]

                for idx, g in candidates.iterrows():
                    g_box = [g['left'], g['top'], g['width'], g['height']]

                    xA = max(p_box[0], g_box[0])
                    yA = max(p_box[1], g_box[1])
                    xB = min(p_box[0]+p_box[2], g_box[0]+g_box[2])
                    yB = min(p_box[1]+p_box[3], g_box[1]+g_box[3])

                    inter = max(0, xB - xA) * max(0, yB - yA)
                    if inter > 0:
                        union = (p_box[2]*p_box[3]) + (g_box[2]*g_box[3]) - inter
                        iou = inter / union
                    else:
                        iou = 0

                    if iou > best_iou:
                        best_iou = iou
                        best_idx = idx

                if best_iou >= IOU_THRESH:
                    v_tp += 1
                    v_gt.at[best_idx, 'matched'] = True
                else:
                    v_fp += 1

        v_fn = len(v_gt) - v_tp

        denom = (2 * v_tp + v_fp + v_fn)
        f1 = (2 * v_tp / denom) if denom > 0 else 0.0

        video_results.append({
            'Video': video,
            'TP': v_tp,
            'FP': v_fp,
            'FN': v_fn,
            'F1': round(f1, 2)
        })

    res_df = pd.DataFrame(video_results)

    if res_df.empty:
        return 0.0

    print("\n" + res_df.to_string(index=False))

    tp, fp, fn = res_df['TP'].sum(), res_df['FP'].sum(), res_df['FN'].sum()

    glob_p = tp / (tp + fp) if (tp + fp) > 0 else 0
    glob_r = tp / (tp + fn) if (tp + fn) > 0 else 0
    glob_f1 = 2 * glob_p * glob_r / (glob_p + glob_r) if (glob_p + glob_r) > 0 else 0

    print(f"GLOBAL METRICS -> P: {glob_p:.4f} | R: {glob_r:.4f} | F1: {glob_f1:.4f}")


all_preds = []

endzone_model_path = "/content/datasets/Dataset/nfl_r2plus1d_Endzone_BEST_F1.pth"
sideline_model_path = "/content/datasets/Dataset/nfl_r2plus1d_Sideline_BEST_F1.pth"


if 'test_samples_endzone' in locals() and len(test_samples_endzone) > 0:
    all_preds.extend(run_inference_with_consensus('Endzone', test_samples_endzone, endzone_model_path))

if 'test_samples_sideline' in locals() and len(test_samples_sideline) > 0:
    all_preds.extend(run_inference_with_consensus('Sideline', test_samples_sideline, sideline_model_path))

if all_preds:
    df_submission = apply_asymmetric_nms(pd.DataFrame(all_preds))
    df_submission.to_csv("submission.csv", index=False)
    print(" File 'submission.csv' saved.")
    evaluate_performance_robust(df_submission, GT_PATH)
    from google.colab import files
    files.download("submission.csv")
else:
    print("Error")